# Setup

In [ ]:
using CSV, D4M, DataFrames, StatsBase, AAUtils, Plots
ENV["LINES"] = 600
ENV["COLUMNS"] = 132
IJulia.set_max_stdio(1 << 20)

## Convienence Constants

In [21]:
csvDir = "/Users/geoffryroberts/cdcWk/synthea/output/ccd/ccda/json/flat/csv"
csvFiles = cd(readdir, csvDir)
TYPE=r"fhirResource\.entry\[\d\]\.resource\.resourceType"
SECTION=r"fhirResource\.entry\[\d*\]\.resource\.section.*"
CODES=r"fhirResource.entry\[\d*\]\.resource.section\[\d*\]\.code\.coding\[\d*\]\.code"
BIRTH=r"fhirResource.entry\[\d\].resource.birthDate,"
OBS_TYPE=r"fhirResource\.entry\[\d*\]\.resource\.resourceType:Observation"
OBS=r"fhirResource\.entry\[\d*\]\.resource\.resourceType\:Observation.*"
MED=r"fhirResource\.entry\[\d*\].resource\.resourceType:Medication"
LOINC=r"coding\[\d*\].code:10160-0"
PRAC="fhirResource.entry[3].resource.id"

### Brutal load

In [22]:
T1 = ReadCSV(csvDir * "/" * csvFiles[1], quotes=false)
T2 = ReadCSV(csvDir * "/" * csvFiles[2], quotes=false)
T3 = ReadCSV(csvDir * "/" * csvFiles[3], quotes=false)
T4 = ReadCSV(csvDir * "/" * csvFiles[4], quotes=false)
T5 = ReadCSV(csvDir * "/" * csvFiles[5], quotes=false)
T6 = ReadCSV(csvDir * "/" * csvFiles[6], quotes=false)
T7 = ReadCSV(csvDir * "/" * csvFiles[7], quotes=false)
T8 = ReadCSV(csvDir * "/" * csvFiles[8], quotes=false)
;

### Consolidate the load

In [23]:
A = T1 + T2 + T3 + T4 + T5 + T6 + T7 + T8
r,c,v = find(A)
display(unique(r))
display("<***>")
display(unique(c))

8-element Array{String,1}:
 "urn:uuid:578a9f82-1927-3a1d-bd43-8f45a2ec9161"
 "urn:uuid:6de60d49-254c-3bdd-a1e0-ddd8a54c2a24"
 "urn:uuid:79c48e91-da28-30fa-8942-c3580db5ad7c"
 "urn:uuid:91b25d65-4406-3862-82a6-10dc806e3243"
 "urn:uuid:97976c69-41d0-3588-8451-1db5a359cbde"
 "urn:uuid:9edc4959-4b38-32c4-ac6c-29a5bcd6ef7a"
 "urn:uuid:c4135e90-7998-34c3-8efe-df2739fabbd6"
 "urn:uuid:d478e93f-aaa6-3c89-bab2-3152c1990ff8"

"<***>"

53557-element Array{String,1}:
 " fhirResource.entry[0].fullUrl:urn:uuid:1ef982ca-ef8b-36f0-9f65-e68af6dccb39"
 "fhirResource.entry[0].fullUrl:urn:uuid:35aadc12-3eb2-3e8b-93bb-4950149ddd73"
 "fhirResource.entry[0].fullUrl:urn:uuid:3ab16ffe-5f59-3c63-8c3c-5d2989975a92"
 "fhirResource.entry[0].fullUrl:urn:uuid:578a9f82-1927-3a1d-bd43-8f45a2ec9161"
 "fhirResource.entry[0].fullUrl:urn:uuid:6de60d49-254c-3bdd-a1e0-ddd8a54c2a24"
 "fhirResource.entry[0].fullUrl:urn:uuid:79c48e91-da28-30fa-8942-c3580db5ad7c"
 "fhirResource.entry[0].fullUrl:urn:uuid:91b25d65-4406-3862-82a6-10dc806e3243"
 "fhirResource.entry[0].fullUrl:urn:uuid:97976c69-41d0-3588-8451-1db5a359cbde"
 "fhirResource.entry[0].fullUrl:urn:uuid:9edc4959-4b38-32c4-ac6c-29a5bcd6ef7a"
 "fhirResource.entry[0].fullUrl:urn:uuid:c4135e90-7998-34c3-8efe-df2739fabbd6"
 "fhirResource.entry[0].fullUrl:urn:uuid:d478e93f-aaa6-3c89-bab2-3152c1990ff8"
 "fhirResource.entry[0].request.method:PUT"
 "fhirResource.entry[0].request.url:Composition/1ef982c

In [24]:
Atype = A[:, TYPE]
r,c,v = find(Atype)
map = countmap(c)
#bar(collect(values(map)))

Dict{String,Int64} with 22 entries:
  "fhirResource.entry[6].resource.resourceType:MedicationStatement" => 8
  "fhirResource.entry[8].resource.resourceType:Medication"          => 7
  "fhirResource.entry[9].resource.resourceType:AllergyIntolerance"  => 4
  "fhirResource.entry[8].resource.resourceType:Condition"           => 8
  "fhirResource.entry[8].resource.resourceType:AllergyIntolerance"  => 4
  "fhirResource.entry[9].resource.resourceType:Condition"           => 8
  "fhirResource.entry[7].resource.resourceType:Condition"           => 8
  "fhirResource.entry[5].resource.resourceType:AllergyIntolerance"  => 8
  "fhirResource.entry[2].resource.resourceType:Device"              => 8
  "fhirResource.entry[9].resource.resourceType:Medication"          => 8
  "fhirResource.entry[3].resource.resourceType:Practitioner"        => 8
  "fhirResource.entry[9].resource.resourceType:MedicationStatement" => 7
  "fhirResource.entry[7].resource.resourceType:AllergyIntolerance"  => 4
  "fhirResource

In [25]:
Acode = A[:, CODES * ":121181"]
printFull(Acode)

3×2 Array{Union{AbstractString, Number},2}:
 ""                                                "fhirResource.entry[0].resource.section[9].code.coding[0].code:121181"
 "urn:uuid:97976c69-41d0-3588-8451-1db5a359cbde"  1.0
 "urn:uuid:c4135e90-7998-34c3-8efe-df2739fabbd6"  1.0

In [28]:
Adocs = getrow(Acode)
B = A[Adocs,:,]
printFull(B)

3×53558 Array{Union{AbstractString, Number},2}:
 ""                                               …   "fhirResource.entry[9].resource.subject.reference:Patient/ff755877-da6d-3feb-b87d-26ded0865eb1"
 "urn:uuid:97976c69-41d0-3588-8451-1db5a359cbde"     1.0
 "urn:uuid:c4135e90-7998-34c3-8efe-df2739fabbd6"     1.0

In [29]:
Aprac = A[:,StartsWith(PRAC)]
printFull(Aprac)

9×10 Array{Union{AbstractString, Number},2}:
 ""                                               …   "fhirResource.entry[3].resource.id:e3802a71-285e-3162-ae22-d55098489f5a"
 "urn:uuid:578a9f82-1927-3a1d-bd43-8f45a2ec9161"     0.0
 "urn:uuid:6de60d49-254c-3bdd-a1e0-ddd8a54c2a24"     0.0
 "urn:uuid:79c48e91-da28-30fa-8942-c3580db5ad7c"     0.0
 "urn:uuid:91b25d65-4406-3862-82a6-10dc806e3243"     0.0
 "urn:uuid:97976c69-41d0-3588-8451-1db5a359cbde"  …  1.0
 "urn:uuid:9edc4959-4b38-32c4-ac6c-29a5bcd6ef7a"     1.0
 "urn:uuid:c4135e90-7998-34c3-8efe-df2739fabbd6"     1.0
 "urn:uuid:d478e93f-aaa6-3c89-bab2-3152c1990ff8"     1.0

In [8]:
B = Acode + Aprac
printFull(B)

9×11 Array{Union{AbstractString, Number},2}:
 ""                                               …   "fhirResource.entry[3].resource.id:e3802a71-285e-3162-ae22-d55098489f5a"
 "urn:uuid:578a9f82-1927-3a1d-bd43-8f45a2ec9161"     0.0
 "urn:uuid:6de60d49-254c-3bdd-a1e0-ddd8a54c2a24"     0.0
 "urn:uuid:79c48e91-da28-30fa-8942-c3580db5ad7c"     0.0
 "urn:uuid:91b25d65-4406-3862-82a6-10dc806e3243"     0.0
 "urn:uuid:97976c69-41d0-3588-8451-1db5a359cbde"  …  1.0
 "urn:uuid:9edc4959-4b38-32c4-ac6c-29a5bcd6ef7a"     1.0
 "urn:uuid:c4135e90-7998-34c3-8efe-df2739fabbd6"     1.0
 "urn:uuid:d478e93f-aaa6-3c89-bab2-3152c1990ff8"     1.0

In [9]:
C = B' * B
printFull(C)

11×11 Array{Union{AbstractString, Number},2}:
 ""                                                                        …   "fhirResource.entry[3].resource.id:e3802a71-285e-3162-ae22-d55098489f5a"
 "fhirResource.entry[0].resource.section[9].code.coding[0].code:121181"       2.0
 "fhirResource.entry[3].resource.id:2525b319-d26f-338a-a210-cdcead4d2cdb"     4.0
 "fhirResource.entry[3].resource.id:27c53ecf-28b6-3fd7-8d7f-5b7e0ac628b3"     4.0
 "fhirResource.entry[3].resource.id:6a0122db-6343-3c96-99ff-49c6bd9ed93b"     3.0
 "fhirResource.entry[3].resource.id:93365189-ca96-3d8c-892b-25305ba33e18"  …  4.0
 "fhirResource.entry[3].resource.id:9dafb442-43f8-344c-b759-6678763bf918"     4.0
 "fhirResource.entry[3].resource.id:9f20e6e5-7d24-3e1c-a150-0831973a409b"     8.0
 "fhirResource.entry[3].resource.id:ccbbd0da-8a2a-3e2a-9a61-36756ce71e67"     3.0
 "fhirResource.entry[3].resource.id:d4d23085-6f2f-31e0-a61c-5c97bbb8bff2"     4.0
 "fhirResource.entry[3].resource.id:e3802a71-285e-3162-ae22-d550